In [4]:
import os
os.chdir( "/home/lucasfuzato/CODING/WINE" )

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from itertools import product

from src.aux_fun import *

tests_df = pd.DataFrame( columns = [ "max_depth" , "algo" ] )

depths = [ 5 , 10 , 15 , 20 , 25 , 30 ]
algos  = [ "gini" , "entropy" ]
for i , ( max_depth , algo ) in enumerate( product( depths , algos ) ):
    tests_df.loc[ i ] = pd.Series( [ max_depth , algo ] , [ "max_depth" , "algo" ] )
print( tests_df ) 

   max_depth     algo
0          5     gini
1          5  entropy
2         10     gini
3         10  entropy
4         15     gini
5         15  entropy
6         20     gini
7         20  entropy
8         25     gini
9         25  entropy
10        30     gini
11        30  entropy


In [5]:
from sklearn.tree import DecisionTreeClassifier as dec_tree
seed_val = 47

X , y , X_2d , columns = class_get_data()

scores = [ "f1_score" , "precision" , "recall" ]
metrics = [ "mean" , "std" ]
data = { tup:list() for tup in product( scores , metrics ) }

for simu_id , ( max_depth , algo ) in enumerate( product( depths , algos ) ):

    lst = []
    fold_iter = get_prepared_data( X , y , seed = seed_val , smote = False )
    for ( X_train , y_train ) , ( X_test , y_test ) in fold_iter:

        tree = dec_tree( criterion = algo , max_depth = max_depth )
        tree.fit( X_train , y_train )

        ts_cls = tree.predict( X_test )
        ts_f1 , prec , recall = bin_f1_score( ts_cls , y_test )

        lst.append( ( ts_f1 , prec , recall ) )
    
    arr = np.array( lst )
    means = arr.mean( axis = 0 )
    data[ ( "f1_score", "mean" ) ].append( means[ 0 ] )
    data[ ( "precision", "mean" ) ].append( means[ 1 ] )
    data[ ( "recall", "mean" ) ].append( means[ 2 ] )

    stds  = arr.std( axis = 0 )
    data[ ( "f1_score", "std" ) ].append( stds[ 0 ] )
    data[ ( "precision", "std" ) ].append( stds[ 1 ] )
    data[ ( "recall", "std" ) ].append( stds[ 2 ] )

final_data = pd.DataFrame( data )
print( final_data )

final_data.to_csv( "dec_tree.csv" , sep = ";" )

    f1_score           precision              recall          
        mean       std      mean       std      mean       std
0   0.537653  0.042240  0.590296  0.044596  0.379724  0.064084
1   0.556881  0.042983  0.631308  0.049114  0.333600  0.075862
2   0.593155  0.048487  0.608410  0.050682  0.547392  0.051119
3   0.561027  0.027651  0.578332  0.025964  0.509110  0.054124
4   0.601937  0.042171  0.597911  0.036770  0.614013  0.066701
5   0.606121  0.033289  0.603741  0.034471  0.613263  0.057745
6   0.616271  0.033169  0.610506  0.032743  0.633569  0.054724
7   0.633307  0.019727  0.628789  0.025269  0.646863  0.031372
8   0.609153  0.026848  0.603614  0.022146  0.625769  0.054281
9   0.617674  0.012848  0.613951  0.019896  0.628845  0.018988
10  0.617986  0.022875  0.612005  0.016876  0.635931  0.052159
11  0.623391  0.017139  0.618708  0.023070  0.637438  0.033119

